**Operations Research in Action &#x25aa; Fall 2024**

# Project 2 &ndash; Exploratory Data Analysis

Before we begin modeling, we need to understand the data that we have, conduct an exploratory analysis to understand the underlying relationships, and if necessary, wrangle the data into the right form for modeling.

## Import the data 

In the same folder as this notebook, there are three data sets, `data/player_data.csv`, `data/stint_data_wide.csv`, and `data/stint_data_long.csv`. Import each data set into a Pandas DataFrame. 

## Describe the data

Recall that the data section of your report should describe each data set. For tabular data, this includes the size of the data, as well as what each row and column represents. The data section also should report basic summary statistics for key variables of interest. 

Examine the three data sets you imported above. How big is the data set? What does each row represent? For each row, what information does the data set contain? Compute basic summary statistics (e.g., mean, standard deviation, minimum, maximum) for some key variables of interest.

## Data wrangling

Recall that Ming wants to assign a value to each player based on plus-minus, a primitive performance measure based on stints. As the case describes, the plus-minus for each stint is

$$
\text{PM}_{\text{stint}} = \frac{\text{HomeGoals}_{\text{stint}} - \text{AwayGoals}_{\text{stint}}}{\text{Duration}_{\text{stint}}}
$$

Wrangle the data to compute the plus-minus for each stint.

We could do something similar to compute a plus-minus for each player:

$$
\text{PM}_{\text{player}} = \frac{\displaystyle\sum_{\text{stint: player plays during stint}} (\text{HomeGoals}_{\text{stint}} - \text{AwayGoals}_{\text{stint}}) \cdot \text{HomeAway}_{\text{player}, \text{stint}}}{\displaystyle\sum_{\text{stint: player plays during stint}} \text{Duration}_{\text{stint}}} 
$$

where

$$
\text{HomeAway}_{\text{player}, \text{stint}} = \begin{cases}
+1 & \text{if player is on the home team during stint} \\
-1 & \text{if player is on the away team during stint}
\end{cases}
$$

## Exploratory data analysis

Exploratory data analysis helps us uncover patterns and identify the underlying relationships in the data.
Ultimately, our goal is to produce analysis and visualizations that provide insight into our formal modeling process (e.g., regression, optimization, simulation).

What insights can you gather about the stint plus-minus ratings? What about the player plus-minus ratings?